SparkSession introduced in version 2.0, It is an entry point to underlying PySpark functionality in order to programmatically create PySpark RDD, DataFrame. It’s object spark is default available in pyspark-shell and it can be created programmatically using SparkSession. 

In [22]:
import pyspark
from pyspark.sql import SparkSession
# instantiate a spark Session
spark = SparkSession.builder.master("local[*]") \
                    .appName('SparkByExamples.com') \
                    .getOrCreate()

Accumulator : are write-only and initialize once variables where only tasks that are running on workers are allowed to update and updates from the workers get propagated automatically to the driver program. But, only the driver program is allowed to access the Accumulator variable using the value property.

In [4]:
rdd=spark.sparkContext.parallelize([1,2,3,4,5])
accuSum=spark.sparkContext.accumulator(0)
def countFun(x):
    global accuSum
    accuSum+=x
rdd.foreach(countFun)
print(accuSum.value)

15


Spark RDD repartition() is used to increase or decrease the RDD/DataFrame partitions whereas the PySpark coalesce() is used to only decrease the number of partitions in an efficient way. 

In [13]:
# In RDD, you can create parallelism at the time of the creation of an RDD using parallelize(), textFile() and wholeTextFiles().
rdd1 = spark.sparkContext.parallelize(list(range(0,20)),6)
print("parallelize " +str(rdd.getNumPartitions()))
rdd1.saveAsTextFile("partition2")

parallelize 6


In [14]:
rdd2 = rdd1.repartition(4)
print("Repartition size : "+str(rdd2.getNumPartitions()))
rdd2.saveAsTextFile("re-partition2")

rdd3 = rdd1.coalesce(4)
print("Repartition size : "+str(rdd3.getNumPartitions()))
rdd3.saveAsTextFile("coalesce2")

# --> Key Difference : coalesce() is optimized or improved version of repartition() where the movement of the data across the partitions is lower using 

Repartition size : 4
Repartition size : 4


 Broadcast variables are read-only shared variables that are cached and available on all nodes in a cluster in-order to access or use by the tasks. Instead of sending this data along with every task, PySpark distributes broadcast variables to the workers using efficient broadcast algorithms to reduce communication costs.

In [21]:
broadcastVar = spark.sparkContext.broadcast({0:'0',1:'1'})
broadcastVar.value

{0: '0', 1: '1'}

RDD (Resilient Distributed Dataset) is a fundamental building block of PySpark which is fault-tolerant, immutable distributed collections of objects. Immutable meaning once you create an RDD you cannot change it. Each record in RDD is divided into logical partitions, which can be computed on different nodes of the cluster. <br>
PySpark RDD Benefits: 
1. In-Memory Processing
2. Immutability
3. Fault Tolerance
4. Lazy Evolution
5. Partitioning

RDD transformations – Transformations are lazy operations, instead of updating an RDD, these operations return another RDD. Transformations are lazy meaning they don’t execute until you call an action on RDD.


In [26]:
rdd = spark.sparkContext.textFile("test.txt")

flatMap()transformation flattens the RDD after applying the function and returns a new RDD. On the below example, first, it splits each record by space in an RDD and finally flattens it. Resulting RDD consists of a single word on each record.

In [32]:
rdd2 = rdd.flatMap(lambda x: x.split(" "))

map() transformation is used the apply any complex operations like adding a column, updating a column e.t.c, the output of map transformations would always have the same number of records as input. <br>In our word count example, we are adding a new column with value 1 for each word, the result of the RDD is PairRDDFunctions which contains key-value pairs, word of type String as Key and 1 of type Int as value.

In [33]:
rdd3 = rdd2.map(lambda x: (x,1))

reduceByKey() merges the values for each key with the function specified. <br> In our example, it reduces the word string by applying the sum function on value. The result of our RDD contains unique words and their count. 

In [37]:
rdd4 = rdd3.reduceByKey(lambda a,b: a+b)

sortByKey() transformation is used to sort RDD elements on key.<br> In our example, first, we convert RDD[(String,Int]) to RDD[(Int, String]) using map transformation and apply sortByKey which ideally does sort on an integer value.

In [42]:
rdd5 = rdd4.map(lambda x: (x[1],x[0])).sortByKey()

filter() transformation is used to filter the records in an RDD. <br> In our example we are filtering all words starts with “a”.

In [58]:
rdd6 = rdd3.filter(lambda x : 'a' in x[0])

RDD actions – operations that trigger computation and return RDD values to a driver program.

count() – Returns the number of records in an RDD

In [60]:
print("Count : "+str(rdd6.count()))

Count : 144


first() – Returns the first record.

In [62]:
firstRec = rdd5.first()
print("First Record : "+str(firstRec[0]) + ","+ firstRec[1])

First Record : 9,Project


max() – Returns max record.

In [64]:
datMax = rdd5.max()
print("Max Record : "+str(datMax[0]) + ","+ datMax[1])

Max Record : 27,with


reduce() – Reduces the records to single, we can use this to count or sum.

In [75]:
# Action - reduce
totalWordCount = rdd5.reduce(lambda a,b: (a[0]+b[0],a[1]+b[1]))
print("dataReduce Record : "+str(totalWordCount[0]))

dataReduce Record : 522


take() – Returns the record specified as an argument.

In [76]:
data3 = rdd6.take(3)

In [77]:
data3

[('Wonderland', 1), ('Carroll', 1), ('anyone', 1)]

Pair RDD is a key-value pair This is mostly used RDD type, 
1. ShuffledRDD 
2. DoubleRDD 
3. SequenceFileRDD  
4. HadoopRDD 
5. ParallelCollectionRDD 


**Shuffling** is a mechanism PySpark uses to redistribute the data across different executors and even across machines. PySpark shuffling triggers when we perform certain transformation operations like gropByKey(), reduceByKey(), join() on RDDS